In [1]:
import sklearn as sk
import numpy as np
import pandas as pd
!pip install thefuzz

In [2]:
df=pd.read_csv("combined.csv")

The dataset gets examined and analyzed.

In [3]:
df.head()

,Type,Price,Bedrooms,Bathrooms,Area,Furnished,Level,Compound,Payment_Option,Delivery_Date,Delivery_Term,City,Ad_Date,Scraping_Date
0,Apartment,1980000,3,3,148.0,No,3,Nyoum,Cash or Installment,2024,Semi Finished,Asyut City,5 days ago,2023-03-13
1,Apartment,4724000,2,2,120.0,Unknown,3,Sun Capital,Cash or Installment,2025,Finished,6th of October,4 days ago,2023-03-13
2,Apartment,355000,3,2,109.0,Unknown,2,Unknown,Unknown Payment,soon,Unknown,Zahraa Al Maadi,13 hours ago,2023-03-13
3,Apartment,3000000,2,2,128.0,No,2,Bloomfields,Cash or Installment,soon,Not Finished,New Cairo - El Tagamoa,4 days ago,2023-03-13
4,Apartment,2900000,2,2,120.0,Unknown,1,Sun Capital,Installment,Ready to move,Unknown,6th of October,4 days ago,2023-03-13


In [4]:
df.shape

(15311, 14)

In [5]:
df.duplicated().sum() #counting duplicates

2792

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15311 entries, 0 to 15310
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Type            15311 non-null  object
 1   Price           15306 non-null  object
 2   Bedrooms        15310 non-null  object
 3   Bathrooms       15310 non-null  object
 4   Area            15300 non-null  object
 5   Furnished       15311 non-null  object
 6   Level           15311 non-null  object
 7   Compound        15311 non-null  object
 8   Payment_Option  15311 non-null  object
 9   Delivery_Date   15311 non-null  object
 10  Delivery_Term   15311 non-null  object
 11  City            12574 non-null  object
 12  Ad_Date         12583 non-null  object
 13  Scraping_Date   15311 non-null  object
dtypes: object(14)
memory usage: 1.6+ MB


In [7]:
df.describe()

,Type,Price,Bedrooms,Bathrooms,Area,Furnished,Level,Compound,Payment_Option,Delivery_Date,Delivery_Term,City,Ad_Date,Scraping_Date
count,15311,15306,15310,15310,15300,15311,15311,15311,15311,15311,15311,12574,12583,15311
unique,5,3416,9,15,369,3,14,250,4,11,5,273,106,14
top,Apartment,Unknown,3,2,Unknown,No,2,Unknown,Cash or Installment,Unknown,Finished,New Cairo - El Tagamoa,2 hours ago,2023-03-21
freq,10985,2728,7968,4879,2728,8567,3586,8952,5260,6093,5241,4601,1959,1121


In [8]:
df.isna().sum()

,0
Type,0
Price,5
Bedrooms,1
Bathrooms,1
Area,11
Furnished,0
Level,0
Compound,0
Payment_Option,0
Delivery_Date,0


Dropping Ad_Time abd Scraping time. Ad_time has not much information and Scraping_time leads to problems in regressions and trainings due to format.

In [9]:
df = df.drop(columns=['Ad_Date', 'Scraping_Date'])

In [10]:
value_counts = df['City'].value_counts()

print("\nCount of unique values:")
print(value_counts)


Count of unique values:
City
New Cairo - El Tagamoa    4601
6th of October            1092
Sheikh Zayed               911
Madinaty                   837
New Capital City           733
                          ... 
Dekheila                     1
Imbaba                       1
Zohour District              1
Capital Gardens              1
Entrada                      1
Name: count, Length: 273, dtype: int64


In [11]:
value_counts = df['Compound'].value_counts()

print("\nCount of unique values:")
print(value_counts)


Count of unique values:
Compound
Unknown                8952
Madinaty                559
Taj City                411
Mountain View iCity     406
Not in Compound         231
                       ... 
Cleo East                 1
Moon City                 1
GCR Golf City             1
Leila                     1
Trevi                     1
Name: count, Length: 250, dtype: int64


Since the City and Compound columns contains many missing values and values often appear only once, this columns will be dropped.

In [12]:
df = df.drop(columns=['City', 'Compound'])

In [13]:
df.tail(20)

,Type,Price,Bedrooms,Bathrooms,Area,Furnished,Level,Payment_Option,Delivery_Date,Delivery_Term
15291,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown Payment,Unknown,Unknown
15292,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown Payment,Unknown,Unknown
15293,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown Payment,Unknown,Unknown
15294,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown Payment,Unknown,Unknown
15295,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown Payment,Unknown,Unknown
15296,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown Payment,Unknown,Unknown
15297,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown Payment,Unknown,Unknown
15298,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown Payment,Unknown,Unknown
15299,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown Payment,Unknown,Unknown
15300,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown Payment,Unknown,Unknown


As there can be seen many rows with only the "unknown" value - the CSV file is examined. All rows with only 'unknown' values in all columns are deleted.


In [14]:
df_new = df[:14199]
print(df_new.tail(10))

            Type      Price Bedrooms Bathrooms   Area Furnished   Level  \
14189  Apartment   780000.0        3         2  165.0   Unknown       6   
14190     Duplex  1015000.0        7         6  327.0   Unknown       7   
14191  Apartment  1355000.0        2         2  114.0        No  Ground   
14192  Apartment   590000.0        1         1   79.0        No       7   
14193  Apartment  3387000.0        3         3  173.0        No       3   
14194  Apartment  3512000.0        3         3  175.0        No       2   
14195  Apartment  3660000.0        3         2  160.0        No       1   
14196  Apartment  2937500.0        2         2  150.0        No  Ground   
14197  Penthouse  5179000.0        4         4  220.0        No       5   
14198  Apartment  4160000.0        3         2  162.0        No  Ground   

            Payment_Option  Delivery_Date  Delivery_Term  
14189  Cash or Installment        Unknown       Finished  
14190  Cash or Installment        Unknown       Finished

In [15]:
df_trimmed = df_new.apply(lambda x: x.str.strip() if x.dtype == "object" else x) #remove leading and trailing whitespace

In [16]:
df_trimmed.shape

(14199, 10)

In [17]:
df_trimmed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14199 entries, 0 to 14198
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Type            14199 non-null  object
 1   Price           14194 non-null  object
 2   Bedrooms        14198 non-null  object
 3   Bathrooms       14198 non-null  object
 4   Area            14188 non-null  object
 5   Furnished       14199 non-null  object
 6   Level           14199 non-null  object
 7   Payment_Option  14199 non-null  object
 8   Delivery_Date   14199 non-null  object
 9   Delivery_Term   14199 non-null  object
dtypes: object(10)
memory usage: 1.1+ MB


In [18]:
#every string to lower case
def to_lowercase(x):
    if isinstance(x, str):
        return x.lower()
    return x

In [19]:
df_lowercase = df_trimmed.applymap(to_lowercase)
print(df_lowercase)

            Type      Price Bedrooms Bathrooms   Area Furnished   Level  \
0      apartment    1980000        3         3  148.0        no       3   
1      apartment    4724000        2         2  120.0   unknown       3   
2      apartment     355000        3         2  109.0   unknown       2   
3      apartment    3000000        2         2  128.0        no       2   
4      apartment    2900000        2         2  120.0   unknown       1   
...          ...        ...      ...       ...    ...       ...     ...   
14194  apartment  3512000.0        3         3  175.0        no       2   
14195  apartment  3660000.0        3         2  160.0        no       1   
14196  apartment  2937500.0        2         2  150.0        no  ground   
14197  penthouse  5179000.0        4         4  220.0        no       5   
14198  apartment  4160000.0        3         2  162.0        no  ground   

            Payment_Option  Delivery_Date  Delivery_Term  
0      cash or installment           202

<ipython-input-19-b423b6d81d5b>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_lowercase = df_trimmed.applymap(to_lowercase)


In [20]:
#TheFuzz library is used to see if there are string mismatches
from thefuzz import fuzz
from thefuzz import process

The unique function was used to make sure, there were no misspellings. For demonstation, it is used for 'type'

In [21]:
# Identify the most common values for 'City' and 'Compound' columns
common_cities = df_lowercase['Type'].value_counts().index.tolist()

# Function to correct misspelled words using thefuzz
def correct_spelling_fuzz(value, correct_list, score_threshold=90):
    if isinstance(value, str):
        # Find the closest match
        best_match = process.extractOne(value, correct_list)
        # If the similarity score is above the threshold, correct the spelling
        if best_match and best_match[1] >= score_threshold:
            return best_match[0]
    return value

# Create a new dataframe df2 by copying df_lowercase
df2 = df_lowercase.copy()

# Apply the correction function to the 'City' and 'Compound' columns in df2
df2['Type'] = df2['Type'].apply(lambda x: correct_spelling_fuzz(x, common_cities))


In [22]:
df_lowercase_count = df_lowercase['Type'].nunique()
print(df_lowercase_count)

5


In [23]:
unique_type_count = df2['Type'].nunique()
print(unique_type_count)

5


As it can be seen, there were no misspellings in the column.

In [24]:
#drops rows that are identical (when all values in all the columns are the same)
df3 = df2.drop_duplicates()

In [25]:
df3.shape

(11333, 10)

In [26]:
#Mixed data types check
!pip install deepchecks

In [27]:
from deepchecks.tabular import Dataset
from deepchecks.tabular.checks import MixedDataTypes

In [28]:
categorical_features = ['Type', 'Furnished', 'Level',
                        'Payment_Option']

#get a dataset out of cat. features

dataset = Dataset(df3, label=None, cat_features=categorical_features)

mixed_types_check = MixedDataTypes().run(dataset)

mixed_types_check.show()

<!--
 ~ ----------------------------------------------------------------------------
 ~ Copyright (C) 2021-2023 Deepchecks (https://www.deepchecks.com)
 ~
 ~ This file is part of Deepchecks.
 ~ Deepchecks is distributed under the terms of the GNU Affero General
 ~ Public License (version 3 or later).
 ~ You should have received a copy of the GNU Affero General Public License
 ~ along with Deepchecks. If not, see .
 ~ ----------------------------------------------------------------------------
 ~
-->
<!DOCTYPE html>
 
 
 
 Mixed Data Types

In [29]:
check = MixedDataTypes().add_condition_rare_type_ratio_not_in_range((0.01, 0.2))
result = check.run(df3)
result.show(show_additional_outputs=False)

deepchecks - WARNING - Received a "pandas.DataFrame" instance. It is recommended to pass a "deepchecks.tabular.Dataset" instance by initializing it with the data and metadata, for example by doing "Dataset(dataframe, label=label, cat_features=cat_features)"
deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 8 categorical features were inferred.: Type, Bedrooms, Bathrooms, Furnished, Level, Payment_Option, Delivery_Date... For full list use dataset.cat_features


<!--
 ~ ----------------------------------------------------------------------------
 ~ Copyright (C) 2021-2023 Deepchecks (https://www.deepchecks.com)
 ~
 ~ This file is part of Deepchecks.
 ~ Deepchecks is distributed under the terms of the GNU Affero General
 ~ Public License (version 3 or later).
 ~ You should have received a copy of the GNU Affero General Public License
 ~ along with Deepchecks. If not, see .
 ~ ----------------------------------------------------------------------------
 ~
-->
<!DOCTYPE html>
 
 
 
 Mixed Data Types

In [30]:
df3.Delivery_Date.unique() #the mixed data is correct

array(['2024', '2025', 'soon', 'ready to move', 'unknown', '2026', '2027',
       '2023', 'within 6 months', '2022', '2028'], dtype=object)

As for the training, it is easier to handle ordinal data than treating them as categorial. Therefor "Level" and "Delivery_Date" will be tranformed into ordinal data - as they are naturally

In [31]:
import numpy as np
mapping = {
    'ready to move': 0,
    'soon': 1,
    'within 6 months': 2,
    '2022': 3,
    '2023': 4,
    '2024': 5,
    '2025': 6,
    '2026': 7,
    '2027': 8,
    '2028': 9,
    'unknown': np.nan
}

# Apply the mapping to the column
df3['Delivery_Date1'] = df3['Delivery_Date'].map(mapping)

# Drop the original column
df3 = df3.drop(columns=['Delivery_Date'])

# Display the results
print(df3)

            Type      Price Bedrooms Bathrooms   Area Furnished   Level  \
0      apartment    1980000        3         3  148.0        no       3   
1      apartment    4724000        2         2  120.0   unknown       3   
2      apartment     355000        3         2  109.0   unknown       2   
3      apartment    3000000        2         2  128.0        no       2   
4      apartment    2900000        2         2  120.0   unknown       1   
...          ...        ...      ...       ...    ...       ...     ...   
14194  apartment  3512000.0        3         3  175.0        no       2   
14195  apartment  3660000.0        3         2  160.0        no       1   
14196  apartment  2937500.0        2         2  150.0        no  ground   
14197  penthouse  5179000.0        4         4  220.0        no       5   
14198  apartment  4160000.0        3         2  162.0        no  ground   

            Payment_Option  Delivery_Term  Delivery_Date1  
0      cash or installment  semi finish

In [32]:
df3.Level.unique()

array(['3', '2', '1', 'ground', '4', '8', '6', '5', '10', '10+', '9',
       'highest', '7', 'unknown'], dtype=object)

In [33]:
level_mapping = {
    'ground': 0,
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
    '6': 6,
    '7': 7,
    '8': 8,
    '9': 9,
    '10': 10,
    '10+': 11,
    'highest': 12,
    'unknown': np.nan
}

# mapping on column
df3['Level1'] = df3['Level'].map(level_mapping)

# Drop the original column
df3 = df3.drop(columns=['Level'])


# Display the updated DataFrame
print(df3.head())

        Type    Price Bedrooms Bathrooms   Area Furnished  \
0  apartment  1980000        3         3  148.0        no   
1  apartment  4724000        2         2  120.0   unknown   
2  apartment   355000        3         2  109.0   unknown   
3  apartment  3000000        2         2  128.0        no   
4  apartment  2900000        2         2  120.0   unknown   

        Payment_Option  Delivery_Term  Delivery_Date1  Level1  
0  cash or installment  semi finished             5.0     3.0  
1  cash or installment       finished             6.0     3.0  
2      unknown payment        unknown             1.0     2.0  
3  cash or installment   not finished             1.0     2.0  
4          installment        unknown             0.0     1.0  


The column values are analysed

In [34]:
df3.Bathrooms.unique()

array(['3', '2', '1', '5', '4', '6', 'unknown', '7', '10', '2.0', '3.0',
       '1.0', '4.0', '5.0', '6.0', nan], dtype=object)

In [35]:
df3.Bedrooms.unique()

array(['3', '2', '1', '4', '5', '6', 'unknown', nan, '7', '9'],
      dtype=object)

In [36]:
columns_to_convert = ['Price', 'Bedrooms', 'Bathrooms', 'Area']

# converting into float, no changing of NaNs
df3['Bedrooms'] = pd.to_numeric(df3['Bedrooms'], errors='coerce')
df3['Bathrooms'] = pd.to_numeric(df3['Bathrooms'], errors='coerce')
df3['Area'] = pd.to_numeric(df3['Area'], errors='coerce')
df3['Price'] = pd.to_numeric(df3['Price'], errors='coerce')

In [37]:
df3.Bedrooms.unique()

array([ 3.,  2.,  1.,  4.,  5.,  6., nan,  7.,  9.])

In [38]:
df3.Price.isna().sum()

6

In [39]:
df3 = df3.dropna(subset=['Price']) # rows with NaNs in Price are deleated

In [40]:
df3.Price.isna().sum()

0

In [41]:
print(df3)

            Type      Price  Bedrooms  Bathrooms   Area Furnished  \
0      apartment  1980000.0       3.0        3.0  148.0        no   
1      apartment  4724000.0       2.0        2.0  120.0   unknown   
2      apartment   355000.0       3.0        2.0  109.0   unknown   
3      apartment  3000000.0       2.0        2.0  128.0        no   
4      apartment  2900000.0       2.0        2.0  120.0   unknown   
...          ...        ...       ...        ...    ...       ...   
14194  apartment  3512000.0       3.0        3.0  175.0        no   
14195  apartment  3660000.0       3.0        2.0  160.0        no   
14196  apartment  2937500.0       2.0        2.0  150.0        no   
14197  penthouse  5179000.0       4.0        4.0  220.0        no   
14198  apartment  4160000.0       3.0        2.0  162.0        no   

            Payment_Option  Delivery_Term  Delivery_Date1  Level1  
0      cash or installment  semi finished             5.0     3.0  
1      cash or installment       fi

In [42]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11327 entries, 0 to 14198
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Type            11327 non-null  object 
 1   Price           11327 non-null  float64
 2   Bedrooms        11326 non-null  float64
 3   Bathrooms       11326 non-null  float64
 4   Area            11316 non-null  float64
 5   Furnished       11327 non-null  object 
 6   Payment_Option  11327 non-null  object 
 7   Delivery_Term   11327 non-null  object 
 8   Delivery_Date1  8271 non-null   float64
 9   Level1          11327 non-null  float64
dtypes: float64(6), object(4)
memory usage: 973.4+ KB


In [43]:
df3.replace('unknown', np.nan, inplace=True)
df3.replace('unknown payment', np.nan, inplace=True)

In [44]:
#checking for special characters
from deepchecks.tabular.checks import SpecialCharacters

In [45]:
dataset = Dataset(df3, label=None)

special_characters_check = SpecialCharacters().run(dataset)

special_characters_check.show()

deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 8 categorical features were inferred.: Type, Bedrooms, Bathrooms, Furnished, Payment_Option, Delivery_Term, Delivery_Date1... For full list use dataset.cat_features


<!--
 ~ ----------------------------------------------------------------------------
 ~ Copyright (C) 2021-2023 Deepchecks (https://www.deepchecks.com)
 ~
 ~ This file is part of Deepchecks.
 ~ Deepchecks is distributed under the terms of the GNU Affero General
 ~ Public License (version 3 or later).
 ~ You should have received a copy of the GNU Affero General Public License
 ~ along with Deepchecks. If not, see .
 ~ ----------------------------------------------------------------------------
 ~
-->
<!DOCTYPE html>
 
 
 
 Special Characters

Next step: fill NaNs for one hot encoding

*   for numeric: skewness is ok, outliers are deleted - therefore mean imputation will be used
*   for other columns: most_frequent-strategy

In [46]:
#calculate skewness to see if the mean or median imputer should be used

from scipy.stats import skew
df4 = df3.dropna()


bedrooms_skewness = skew(df4['Bedrooms'].dropna())
bathrooms_skewness = skew(df4['Bathrooms'].dropna())
area_skewness = skew(df4['Area'].dropna())

print(f"Skewness from 'Bedrooms' column: {bedrooms_skewness}")
print(f"Skewness from 'Bathrooms' column: {bathrooms_skewness}")
print(f"Skewness from 'Area' column: {area_skewness}")

Skewness from 'Bedrooms' column: -0.02548385679558974
Skewness from 'Bathrooms' column: 0.07774326376413791
Skewness from 'Area' column: 2.216206989317493


The skewness from 'Bedrooms' and 'Bathrooms' is ok, but in 'Area' it's positively skewed.

In [47]:
# checking for outliers
from deepchecks.tabular.checks import OutlierSampleDetection

In [48]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11327 entries, 0 to 14198
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Type            11327 non-null  object 
 1   Price           11327 non-null  float64
 2   Bedrooms        11326 non-null  float64
 3   Bathrooms       11326 non-null  float64
 4   Area            11316 non-null  float64
 5   Furnished       7980 non-null   object 
 6   Payment_Option  10371 non-null  object 
 7   Delivery_Term   9244 non-null   object 
 8   Delivery_Date1  8271 non-null   float64
 9   Level1          11327 non-null  float64
dtypes: float64(6), object(4)
memory usage: 973.4+ KB


In [49]:
from deepchecks.tabular import Dataset
dataset = Dataset(df3, label='Price', cat_features=['Type', 'Bedrooms', 'Bathrooms', 'Furnished', 'Payment_Option', 'Delivery_Term'])
check = OutlierSampleDetection()
check.add_condition_outlier_ratio_less_or_equal(max_outliers_ratio=0.01, outlier_score_threshold=0.8)
check.run(dataset)

<!--
 ~ ----------------------------------------------------------------------------
 ~ Copyright (C) 2021-2023 Deepchecks (https://www.deepchecks.com)
 ~
 ~ This file is part of Deepchecks.
 ~ Deepchecks is distributed under the terms of the GNU Affero General
 ~ Public License (version 3 or later).
 ~ You should have received a copy of the GNU Affero General Public License
 ~ along with Deepchecks. If not, see .
 ~ ----------------------------------------------------------------------------
 ~
-->
<!DOCTYPE html>
 
 
 
 Outlier Sample Detection

As there are outliers, they are calculated with the robust z-score. Here, the Median Absolute Deviation (MAD) and median will be used.
The median is chosen over the mean because it is more robust against outliers. The MAD is a robust measure of statistical dispersion. Unlike standard deviation, which can be heavily influenced by outliers, the MAD measures the average distance of data points from the median.

In [50]:
from scipy.stats import median_abs_deviation


numeric_cols = df3.select_dtypes(include=[np.number])

# Calculate the median and MAD for numeric columns
median = numeric_cols.median()
mad = numeric_cols.apply(median_abs_deviation)

# Calculation of z-score
z_scores = 0.6745 * (numeric_cols - median) / mad  # mit normalizing factor scores comparable to the standard z-scores


print(z_scores)

# rows which have Z-scores beyond the threshold
outliers_count = (np.abs(z_scores) > 3.5).sum(axis=1) #3.5 is common for robust z-scores
print(outliers_count.value_counts())

rows_to_keep = outliers_count == 0

# Filter the original DataFrame to keep only these rows
df3_cleaned = df3[rows_to_keep]

# Optionally, print the number of rows before and after filtering
print(f"Original number of rows: {df3.shape[0]}")
print(f"Number of rows after removing outliers: {df3_cleaned.shape[0]}")

          Price  Bedrooms  Bathrooms  Area  Delivery_Date1  Level1
0     -0.183079       NaN        NaN   NaN             NaN  0.6745
1      1.138941       NaN        NaN   NaN             NaN  0.6745
2     -0.965980       NaN        NaN   NaN             NaN  0.0000
3      0.308343       NaN        NaN   NaN             NaN  0.0000
4      0.260164       NaN        NaN   NaN             NaN -0.6745
...         ...       ...        ...   ...             ...     ...
14194  0.555017       NaN        NaN   NaN             NaN  0.0000
14195  0.626321       NaN        NaN   NaN             NaN -0.6745
14196  0.278231       NaN        NaN   NaN             NaN -1.3490
14197  1.358154       NaN        NaN   NaN             NaN  2.0235
14198  0.867214       NaN        NaN   NaN             NaN -1.3490

[11327 rows x 6 columns]
0    10537
1      766
2       24
Name: count, dtype: int64
Original number of rows: 11327
Number of rows after removing outliers: 10537


Before imputation: split dataset

In [51]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df3_cleaned, test_size=0.25)

X_test = test_df.drop(columns=['Price'])
y_test_1 = test_df['Price']

In [52]:
train_df.head()

,Type,Price,Bedrooms,Bathrooms,Area,Furnished,Payment_Option,Delivery_Term,Delivery_Date1,Level1
971,apartment,2953000.0,3.0,3.0,175.0,no,cash,semi finished,0.0,1.0
11711,apartment,600000.0,3.0,3.0,160.0,no,cash or installment,core & shell,0.0,3.0
8017,apartment,1953000.0,3.0,3.0,200.0,no,cash or installment,not finished,7.0,1.0
13899,apartment,870000.0,3.0,1.0,120.0,no,cash,finished,NaN,2.0
8159,apartment,4700000.0,1.0,1.0,81.0,NaN,NaN,finished,1.0,3.0


In [53]:
X_test.head()

,Type,Bedrooms,Bathrooms,Area,Furnished,Payment_Option,Delivery_Term,Delivery_Date1,Level1
5655,apartment,4.0,4.0,220.0,no,cash or installment,semi finished,0.0,3.0
1178,apartment,3.0,2.0,143.0,no,cash or installment,finished,0.0,0.0
13281,apartment,3.0,3.0,168.0,no,cash or installment,semi finished,5.0,3.0
12818,apartment,3.0,2.0,103.0,no,cash,finished,0.0,2.0
8512,apartment,3.0,3.0,145.0,NaN,cash or installment,NaN,0.0,2.0


In [54]:
train_df.Payment_Option.unique()

array(['cash', 'cash or installment', nan, 'installment'], dtype=object)

In [55]:
'unknown' in train_df['Price'].values

False

In [56]:
'unknown' in test_df['Price'].values

False

In [57]:
train_df.isna().sum()

,0
Type,0
Price,0
Bedrooms,0
Bathrooms,0
Area,7
Furnished,2320
Payment_Option,648
Delivery_Term,1480
Delivery_Date1,2132
Level1,0


In [58]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7902 entries, 971 to 3964
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Type            7902 non-null   object 
 1   Price           7902 non-null   float64
 2   Bedrooms        7902 non-null   float64
 3   Bathrooms       7902 non-null   float64
 4   Area            7895 non-null   float64
 5   Furnished       5582 non-null   object 
 6   Payment_Option  7254 non-null   object 
 7   Delivery_Term   6422 non-null   object 
 8   Delivery_Date1  5770 non-null   float64
 9   Level1          7902 non-null   float64
dtypes: float64(6), object(4)
memory usage: 679.1+ KB


In [59]:
#if the type gets lost

columns_to_convert = ['Price', 'Bedrooms', 'Bathrooms', 'Area']

for column in columns_to_convert:
    train_df[column] = train_df[column].astype(float)




In [60]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7902 entries, 971 to 3964
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Type            7902 non-null   object 
 1   Price           7902 non-null   float64
 2   Bedrooms        7902 non-null   float64
 3   Bathrooms       7902 non-null   float64
 4   Area            7895 non-null   float64
 5   Furnished       5582 non-null   object 
 6   Payment_Option  7254 non-null   object 
 7   Delivery_Term   6422 non-null   object 
 8   Delivery_Date1  5770 non-null   float64
 9   Level1          7902 non-null   float64
dtypes: float64(6), object(4)
memory usage: 679.1+ KB


In [61]:
from sklearn.impute import SimpleImputer
#mean imputation for numerical columns exept Area

def impute_group(group):
    group = group.copy()
    numerical_columns = ['Bedrooms', 'Bathrooms', 'Level1', 'Delivery_Date1']
    num_imputer = SimpleImputer(strategy='mean')
    group[numerical_columns] = num_imputer.fit_transform(group[numerical_columns])
    return group

#group by Type
train_df = train_df.groupby('Type', group_keys=False, as_index=False).apply(impute_group).set_index(train_df.index) #group_keys=False for keeping original index

# median imputation for Area

def impute_group1(group):
    group = group.copy()
    numerical_column = ['Area']
    num_imputer = SimpleImputer(strategy='median')
    group[numerical_column] = num_imputer.fit_transform(group[numerical_column])
    return group

#group by 'Type'
train_df = train_df.groupby('Type', group_keys=False, as_index=False).apply(impute_group1).set_index(train_df.index)

print("\nTrain-DataFrame after median imputation:")
print(train_df)

#### Test-Dataset Transformation ####
numerical_columns = ['Bedrooms', 'Bathrooms', 'Level1', 'Delivery_Date1']

#train the imputer on trainingsdata
num_imputer_mean = SimpleImputer(strategy='mean')
num_imputer_mean.fit(train_df[numerical_columns])

# Transform numerical columns exept area
X_test[numerical_columns] = num_imputer_mean.transform(X_test[numerical_columns])

#train the imputer on trainingsdata
num_imputer_median = SimpleImputer(strategy='median')
num_imputer_median.fit(train_df[['Area']])

# Transform column area
X_test['Area'] = num_imputer_median.transform(X_test[['Area']])

print("\nTest-DataFrame after imputation:")
print(X_test)


Train-DataFrame after median imputation:
            Type      Price  Bedrooms  Bathrooms   Area Furnished  \
971    apartment  2953000.0       3.0        3.0  175.0        no   
11711  apartment   600000.0       3.0        3.0  160.0        no   
8017   apartment  1953000.0       3.0        3.0  200.0        no   
13899  apartment   870000.0       3.0        1.0  120.0        no   
8159   apartment  4700000.0       1.0        1.0   81.0       NaN   
...          ...        ...       ...        ...    ...       ...   
797    apartment   400000.0       2.0        2.0  128.0       NaN   
6843   apartment  2850000.0       3.0        2.0  108.0        no   
6458   apartment   800000.0       3.0        2.0  207.0       NaN   
8532   apartment  2650000.0       3.0        3.0  168.0        no   
3964   apartment  4500000.0       4.0        4.0  200.0       yes   

            Payment_Option  Delivery_Term  Delivery_Date1  Level1  
971                   cash  semi finished        0.000000    

In [62]:
train_df.head()

,Type,Price,Bedrooms,Bathrooms,Area,Furnished,Payment_Option,Delivery_Term,Delivery_Date1,Level1
971,apartment,2953000.0,3.0,3.0,175.0,no,cash,semi finished,0.000000,1.0
11711,apartment,600000.0,3.0,3.0,160.0,no,cash or installment,core & shell,0.000000,3.0
8017,apartment,1953000.0,3.0,3.0,200.0,no,cash or installment,not finished,7.000000,1.0
13899,apartment,870000.0,3.0,1.0,120.0,no,cash,finished,1.640055,2.0
8159,apartment,4700000.0,1.0,1.0,81.0,NaN,NaN,finished,1.000000,3.0


In [63]:
X_test.head()

,Type,Bedrooms,Bathrooms,Area,Furnished,Payment_Option,Delivery_Term,Delivery_Date1,Level1
5655,apartment,4.0,4.0,220.0,no,cash or installment,semi finished,0.0,3.0
1178,apartment,3.0,2.0,143.0,no,cash or installment,finished,0.0,0.0
13281,apartment,3.0,3.0,168.0,no,cash or installment,semi finished,5.0,3.0
12818,apartment,3.0,2.0,103.0,no,cash,finished,0.0,2.0
8512,apartment,3.0,3.0,145.0,NaN,cash or installment,NaN,0.0,2.0


In [64]:
train_df.Bathrooms.unique()

array([3., 1., 2., 4., 5., 6., 7.])

In [65]:
value_counts = train_df['Bedrooms'].value_counts()

print("\nCount of unique values:")
print(value_counts)


Count of unique values:
Bedrooms
3.0    5003
2.0    1855
4.0     681
1.0     278
5.0      78
6.0       6
7.0       1
Name: count, dtype: int64


In [66]:
train_df['Bedrooms'] = train_df['Bedrooms'].round().astype(int)
train_df['Bathrooms'] = train_df['Bathrooms'].round().astype(int)


In [67]:
value_counts = train_df['Bedrooms'].value_counts()

print("\nCount of unique values:")
print(value_counts)


Count of unique values:
Bedrooms
3    5003
2    1855
4     681
1     278
5      78
6       6
7       1
Name: count, dtype: int64


In [68]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7902 entries, 971 to 3964
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Type            7902 non-null   object 
 1   Price           7902 non-null   float64
 2   Bedrooms        7902 non-null   int64  
 3   Bathrooms       7902 non-null   int64  
 4   Area            7902 non-null   float64
 5   Furnished       5582 non-null   object 
 6   Payment_Option  7254 non-null   object 
 7   Delivery_Term   6422 non-null   object 
 8   Delivery_Date1  7902 non-null   float64
 9   Level1          7902 non-null   float64
dtypes: float64(4), int64(2), object(4)
memory usage: 679.1+ KB


### For object-columns using the most-frequent strategy by grouping depending on the type of the properties

In [69]:
#for object-columns using the most-frequent strategy by grouping depending on the type of the properties

# object-columns, where NaNs should be imputed
object_columns = ['Furnished', 'Payment_Option', 'Delivery_Term']

# define, that they should be grouped
def impute_group_mode(group):
    imputer = SimpleImputer(strategy='most_frequent')
    group = group.copy()
    group[object_columns] = imputer.fit_transform(group[object_columns])
    return group

train_df = train_df.groupby('Type', group_keys=False, as_index=False).apply(impute_group_mode).set_index(train_df.index) #group by column 'Type'

print("\nTraining-DataFrame after imputation:")
print(train_df)


####Test-Data transform###

object_columns = ['Furnished', 'Payment_Option', 'Delivery_Term']

#train imputer
cat_imputer_median = SimpleImputer(strategy='most_frequent')
cat_imputer_median.fit(train_df[object_columns])

cat_imputer = SimpleImputer(strategy='most_frequent')
X_test[object_columns] = cat_imputer.fit_transform(X_test[object_columns])

print("\nTest-DataFrame after imputation:")
print(X_test)


Training-DataFrame after imputation:
            Type      Price  Bedrooms  Bathrooms   Area Furnished  \
971    apartment  2953000.0         3          3  175.0        no   
11711  apartment   600000.0         3          3  160.0        no   
8017   apartment  1953000.0         3          3  200.0        no   
13899  apartment   870000.0         3          1  120.0        no   
8159   apartment  4700000.0         1          1   81.0        no   
...          ...        ...       ...        ...    ...       ...   
797    apartment   400000.0         2          2  128.0        no   
6843   apartment  2850000.0         3          2  108.0        no   
6458   apartment   800000.0         3          2  207.0        no   
8532   apartment  2650000.0         3          3  168.0        no   
3964   apartment  4500000.0         4          4  200.0       yes   

            Payment_Option  Delivery_Term  Delivery_Date1  Level1  
971                   cash  semi finished        0.000000     1.0

In [70]:
train_df.head()

,Type,Price,Bedrooms,Bathrooms,Area,Furnished,Payment_Option,Delivery_Term,Delivery_Date1,Level1
971,apartment,2953000.0,3,3,175.0,no,cash,semi finished,0.000000,1.0
11711,apartment,600000.0,3,3,160.0,no,cash or installment,core & shell,0.000000,3.0
8017,apartment,1953000.0,3,3,200.0,no,cash or installment,not finished,7.000000,1.0
13899,apartment,870000.0,3,1,120.0,no,cash,finished,1.640055,2.0
8159,apartment,4700000.0,1,1,81.0,no,cash or installment,finished,1.000000,3.0


In [71]:
X_test.head()

,Type,Bedrooms,Bathrooms,Area,Furnished,Payment_Option,Delivery_Term,Delivery_Date1,Level1
5655,apartment,4.0,4.0,220.0,no,cash or installment,semi finished,0.0,3.0
1178,apartment,3.0,2.0,143.0,no,cash or installment,finished,0.0,0.0
13281,apartment,3.0,3.0,168.0,no,cash or installment,semi finished,5.0,3.0
12818,apartment,3.0,2.0,103.0,no,cash,finished,0.0,2.0
8512,apartment,3.0,3.0,145.0,no,cash or installment,finished,0.0,2.0


In [72]:
train_df.isna().sum()

,0
Type,0
Price,0
Bedrooms,0
Bathrooms,0
Area,0
Furnished,0
Payment_Option,0
Delivery_Term,0
Delivery_Date1,0
Level1,0


In [73]:
X_test.isna().sum()

,0
Type,0
Bedrooms,0
Bathrooms,0
Area,0
Furnished,0
Payment_Option,0
Delivery_Term,0
Delivery_Date1,0
Level1,0


In [74]:
# Find rows where the 'Price' column has NaN
rows_with_nan_in_price = train_df[train_df['Price'].isna()]

# Display the rows
print(rows_with_nan_in_price)

Empty DataFrame
Columns: [Type, Price, Bedrooms, Bathrooms, Area, Furnished, Payment_Option, Delivery_Term, Delivery_Date1, Level1]
Index: []


In [75]:
#check for linearity - one-hot encoding to use linear regression
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.compose import ColumnTransformer



In [76]:
for col in ['Type', 'Furnished', 'Payment_Option', 'Delivery_Term']:
    print(f"{col} unique values: {train_df[col].unique()}")

Type unique values: ['apartment' 'duplex' 'studio' 'penthouse']
Furnished unique values: ['no' 'yes']
Payment_Option unique values: ['cash' 'cash or installment' 'installment']
Delivery_Term unique values: ['semi finished' 'core & shell' 'not finished' 'finished']


In [77]:
categorical_columns = ['Type', 'Furnished', 'Payment_Option', 'Delivery_Term']

# one-hot-encoder (sparse_output=False for dense Array)
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit und transform of cat columns
one_hot_encoded = encoder.fit_transform(train_df[categorical_columns])

#dataframe for encoded cat columns
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns), index=train_df.index)

# concat and drop orig. cat. columns
df_encoded = pd.concat([train_df, one_hot_df], axis=1)
df_encoded = df_encoded.drop(categorical_columns, axis=1)

print(f"Encoded data : \n{df_encoded}")

Encoded data : 
           Price  Bedrooms  Bathrooms   Area  Delivery_Date1  Level1  \
971    2953000.0         3          3  175.0        0.000000     1.0   
11711   600000.0         3          3  160.0        0.000000     3.0   
8017   1953000.0         3          3  200.0        7.000000     1.0   
13899   870000.0         3          1  120.0        1.640055     2.0   
8159   4700000.0         1          1   81.0        1.000000     3.0   
...          ...       ...        ...    ...             ...     ...   
797     400000.0         2          2  128.0        1.000000     2.0   
6843   2850000.0         3          2  108.0        0.000000     3.0   
6458    800000.0         3          2  207.0        1.000000     0.0   
8532   2650000.0         3          3  168.0        0.000000     2.0   
3964   4500000.0         4          4  200.0        0.000000     4.0   

       Type_apartment  Type_duplex  Type_penthouse  Type_studio  Furnished_no  \
971               1.0          0.0    

In [78]:
df_encoded.head()

,Price,Bedrooms,Bathrooms,Area,Delivery_Date1,Level1,Type_apartment,Type_duplex,Type_penthouse,Type_studio,Furnished_no,Furnished_yes,Payment_Option_cash,Payment_Option_cash or installment,Payment_Option_installment,Delivery_Term_core & shell,Delivery_Term_finished,Delivery_Term_not finished,Delivery_Term_semi finished
971,2953000.0,3,3,175.0,0.000000,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
11711,600000.0,3,3,160.0,0.000000,3.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
8017,1953000.0,3,3,200.0,7.000000,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
13899,870000.0,3,1,120.0,1.640055,2.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
8159,4700000.0,1,1,81.0,1.000000,3.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [79]:
X_test.head()

,Type,Bedrooms,Bathrooms,Area,Furnished,Payment_Option,Delivery_Term,Delivery_Date1,Level1
5655,apartment,4.0,4.0,220.0,no,cash or installment,semi finished,0.0,3.0
1178,apartment,3.0,2.0,143.0,no,cash or installment,finished,0.0,0.0
13281,apartment,3.0,3.0,168.0,no,cash or installment,semi finished,5.0,3.0
12818,apartment,3.0,2.0,103.0,no,cash,finished,0.0,2.0
8512,apartment,3.0,3.0,145.0,no,cash or installment,finished,0.0,2.0


In [80]:
###Test-dataset transformation##

categorical_columns = ['Type', 'Furnished',  'Payment_Option', 'Delivery_Term']

#one-hot-encoder transform test data and get dataframe
one_hot_encoded_test = encoder.transform(X_test[categorical_columns])
one_hot_df_test = pd.DataFrame(one_hot_encoded_test, columns=encoder.get_feature_names_out(categorical_columns), index=X_test.index)

#drop orig. catcolumns and concat with one hot encoded df
X_test_non_cat = X_test.drop(columns=categorical_columns)
one_hot_df_test = pd.concat([X_test_non_cat, one_hot_df_test], axis=1)

print("df_encoded:", df_encoded.columns)
print("one_hot_df_test:", one_hot_df_test.columns)

print(one_hot_df_test.head())

df_encoded: Index(['Price', 'Bedrooms', 'Bathrooms', 'Area', 'Delivery_Date1', 'Level1',
       'Type_apartment', 'Type_duplex', 'Type_penthouse', 'Type_studio',
       'Furnished_no', 'Furnished_yes', 'Payment_Option_cash',
       'Payment_Option_cash or installment', 'Payment_Option_installment',
       'Delivery_Term_core & shell', 'Delivery_Term_finished',
       'Delivery_Term_not finished', 'Delivery_Term_semi finished'],
      dtype='object')
one_hot_df_test: Index(['Bedrooms', 'Bathrooms', 'Area', 'Delivery_Date1', 'Level1',
       'Type_apartment', 'Type_duplex', 'Type_penthouse', 'Type_studio',
       'Furnished_no', 'Furnished_yes', 'Payment_Option_cash',
       'Payment_Option_cash or installment', 'Payment_Option_installment',
       'Delivery_Term_core & shell', 'Delivery_Term_finished',
       'Delivery_Term_not finished', 'Delivery_Term_semi finished'],
      dtype='object')
       Bedrooms  Bathrooms   Area  Delivery_Date1  Level1  Type_apartment  \
5655        4.0    

In [81]:
one_hot_df_test.head()

,Bedrooms,Bathrooms,Area,Delivery_Date1,Level1,Type_apartment,Type_duplex,Type_penthouse,Type_studio,Furnished_no,Furnished_yes,Payment_Option_cash,Payment_Option_cash or installment,Payment_Option_installment,Delivery_Term_core & shell,Delivery_Term_finished,Delivery_Term_not finished,Delivery_Term_semi finished
5655,4.0,4.0,220.0,0.0,3.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1178,3.0,2.0,143.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
13281,3.0,3.0,168.0,5.0,3.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
12818,3.0,2.0,103.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
8512,3.0,3.0,145.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [82]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7902 entries, 971 to 3964
Data columns (total 19 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Price                               7902 non-null   float64
 1   Bedrooms                            7902 non-null   int64  
 2   Bathrooms                           7902 non-null   int64  
 3   Area                                7902 non-null   float64
 4   Delivery_Date1                      7902 non-null   float64
 5   Level1                              7902 non-null   float64
 6   Type_apartment                      7902 non-null   float64
 7   Type_duplex                         7902 non-null   float64
 8   Type_penthouse                      7902 non-null   float64
 9   Type_studio                         7902 non-null   float64
 10  Furnished_no                        7902 non-null   float64
 11  Furnished_yes                       7902 non-n

In [83]:
#linear regression for linearity check
#from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LinearRegression
#from sklearn.metrics import r2_score
#from sklearn.preprocessing import StandardScaler


#X = df_encoded.drop(['Price'], axis=1)
#y = df_encoded['Price']

# Splitting into training and test data
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


#scaler = StandardScaler()
#X_train_std = scaler.fit_transform(X_train)
#X_test_std = scaler.transform(X_test)

# model a linear regression
#model = LinearRegression()
#model.fit(X_train_std, y_train)

# predict on trainingsdata
#y_train_pred = model.predict(X_train_std)
#y_test_pred = model.predict(X_test_std)

# calculate the R^2-value
#r2 = r2_score(y_train, y_train_pred)
#print(f'R^2 Score on training data: {r2}')
#r2_test = r2_score(y_test, y_test_pred)
#print(f'R^2 Score on test data: {r2_test}')

#R^2 Score on training data: 0.21467754869269084
#R^2 Score on test data: 0.1999583123693125#

In [84]:
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score


target_column = 'Price'

#only take the numericle columns
numeric_columns = df_encoded.select_dtypes(include=['int64', 'float64']).columns
numeric_columns = numeric_columns.drop(target_column)
df_numeric = df_encoded[numeric_columns]

# set the target
y = df_encoded[target_column]

# Train-Test-Split
X_train, X_test, y_train, y_test = train_test_split(df_numeric, y, test_size=0.2, random_state=42)

# Scaling features
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_stv = sc.fit_transform(X_train)
X_test_stv = sc.transform(X_test)


In [85]:

# Pipeline PCA
#pca_pipeline = Pipeline([
#   ('pca', PCA(n_components=0.95,  svd_solver ='full'))  #the minimum number of principal components are kept so that 95 percent of the variance is retained
#])

# PCA-transformation
#X_train_pca = pca_pipeline.fit_transform(X_train)
#X_test_pca = pca_pipeline.transform(X_test)

# Model-training PCA
#model = LinearRegression()
#model.fit(X_train_pca, y_train)
#pca_score = model.score(X_test_pca, y_test)
#print("R²-Score with PCA: ", pca_score)

# Kernel PCA
#kpca_pipeline = Pipeline([
 #   ('kpca', KernelPCA(n_components = 1200, kernel='poly'))  # RBF-Kernel, 1200 0.454
#])

#Kernel PCA-transformation
#X_train_kpca = kpca_pipeline.fit_transform(X_train)
#X_test_kpca = kpca_pipeline.transform(X_test)

# Model-training and evaluation with KPCA
#model.fit(X_train_kpca, y_train)
#kpca_score = model.score(X_test_kpca, y_test)
#print("R²-Score with Kernel PCA: ", kpca_score)



#R²-Score with PCA:  0.42210162691229003
#R²-Score with Kernel PCA:  0.4821998044699718/0.44733756580779716



In [86]:
df_numeric.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7902 entries, 971 to 3964
Data columns (total 18 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Bedrooms                            7902 non-null   int64  
 1   Bathrooms                           7902 non-null   int64  
 2   Area                                7902 non-null   float64
 3   Delivery_Date1                      7902 non-null   float64
 4   Level1                              7902 non-null   float64
 5   Type_apartment                      7902 non-null   float64
 6   Type_duplex                         7902 non-null   float64
 7   Type_penthouse                      7902 non-null   float64
 8   Type_studio                         7902 non-null   float64
 9   Furnished_no                        7902 non-null   float64
 10  Furnished_yes                       7902 non-null   float64
 11  Payment_Option_cash                 7902 non-n

In [87]:
kpca = KernelPCA(n_components = 1200, kernel = 'poly')
X_train_kpca = kpca.fit_transform(X_train_stv)
X_test_kpca = kpca.transform(X_test_stv)

In [88]:
X_train_df = pd.DataFrame(X_train_kpca).reset_index(drop=True)
X_test_df = pd.DataFrame(X_test_kpca).reset_index(drop=True)

# Convert target to Series
y_train_series = pd.Series(y_train).reset_index(drop=True)
y_test_series = pd.Series(y_test).reset_index(drop=True)

# Concat the datasets
X_combined = pd.concat([X_train_df, X_test_df]).reset_index(drop=True)
y_combined = pd.concat([y_train_series, y_test_series]).reset_index(drop=True)

# Kombiniere X_combined und y_combined zu einem DataFrame
final_combined_dataset = pd.concat([X_combined, y_combined], axis=1)

# Setze die Spaltennamen (0, 1, 2, ..., 1199, 'Price')
column_names = list(range(X_combined.shape[1])) + ['Price']
final_combined_dataset.columns = column_names

# Überprüfen auf NaN-Werte und die endgültige Form
print("NaN-Werte im final_combined_dataset:", final_combined_dataset.isna().sum().sum())
print("final_combined_dataset shape:", final_combined_dataset.shape)
print(final_combined_dataset.head())

NaN-Werte im final_combined_dataset: 0
final_combined_dataset shape: (7902, 1201)
          0         1         2         3         4         5         6  \
0 -0.307090  0.142124 -0.622552 -0.187870 -0.069397 -0.042602 -0.092708   
1 -0.262176 -0.181578 -0.484598 -0.128971  0.016128  0.162254 -0.847383   
2 -0.293466  0.000404 -0.586791 -0.177425 -0.014623  0.040411 -0.495701   
3 -0.383138 -0.181903 -0.368634 -0.024209  0.040738 -0.110291  1.760713   
4 -0.329099 -0.211688 -0.349753 -0.054355  0.029454  0.130720 -0.232214   

          7         8         9  ...  1191  1192  1193  1194  1195  1196  \
0 -0.029137  0.079544 -0.710472  ...  -0.0  -0.0  -0.0  -0.0   0.0   0.0   
1 -0.193351  0.554112  0.461377  ...   0.0   0.0  -0.0  -0.0  -0.0  -0.0   
2 -0.091069 -0.074902  0.506315  ...   0.0   0.0   0.0   0.0  -0.0   0.0   
3 -1.630497  0.848805 -0.177312  ...  -0.0  -0.0   0.0   0.0  -0.0  -0.0   
4 -0.042452 -0.258154  0.583917  ...  -0.0  -0.0  -0.0  -0.0  -0.0   0.0   

   1197  1

In [89]:
###test data transform##
scaler = StandardScaler()
df_numeric1=pd.DataFrame(df_numeric)  #train X_test on all trainingsdata
X_test_final_std = sc.transform(one_hot_df_test)


In [90]:
#kPCA on testdata
X_test_kpca = kpca.transform(X_test_final_std)  # kpca fitted on trainingsdata

In [91]:
# Transform X_test_kpca into DataFrame
df_test_kpca = pd.DataFrame(X_test_kpca,
                            columns=[f'{i}' for i in range(X_test_kpca.shape[1])]) #column-numbers

#restet index, important for upload
df_test_kpca = df_test_kpca.reset_index(drop=True)

# Convert y_test_1 to Series
y_test_1_df = pd.Series(y_test_1).reset_index(drop=True)

# Concat y_test as 'Price'
df_test_kpca['Price'] = y_test_1_df.values

# Output the final test dataset
print('\nFinal test dataset:')
print(df_test_kpca.head())


Final test dataset:
          0         1         2         3         4         5         6  \
0 -0.388425 -0.281695  0.057608  0.087147  0.080330  0.031652  0.313105   
1 -0.337027 -0.175829 -0.390852 -0.054255  0.020693  0.106842 -0.271208   
2 -0.380922 -0.198100 -0.294771 -0.075536  0.036685 -0.017023  0.494882   
3 -0.282685 -0.185793 -0.457106 -0.093244  0.026257  0.168999 -0.712751   
4 -0.341934 -0.204858 -0.316830 -0.067561  0.026563  0.107774 -0.226687   

          7         8         9  ...  1191  1192  1193  1194  1195  1196  \
0  0.138582 -0.738401 -0.336028  ...   0.0   0.0   0.0   0.0   0.0   0.0   
1 -0.014334  0.163809 -0.572390  ...   0.0   0.0   0.0   0.0   0.0   0.0   
2  0.141774 -0.806813 -0.477930  ...   0.0   0.0   0.0   0.0   0.0   0.0   
3 -0.165520  0.431295  0.375553  ...   0.0   0.0   0.0   0.0   0.0   0.0   
4 -0.009070  0.130056 -0.575886  ...   0.0   0.0   0.0   0.0   0.0   0.0   

   1197  1198  1199      Price  
0   0.0   0.0   0.0   400000.0  
1   0

In [92]:
#download
from google.colab import files
final_combined_dataset.to_csv('TrainSet_regression.csv', index=False)
df_test_kpca.to_csv('TestSet_regression.csv', index=False)
files.download('TrainSet_regression.csv')
files.download('TestSet_regression.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>